In [1]:
from flask import Flask, request, jsonify
from PIL import Image
from werkzeug.utils import secure_filename
import os
import tempfile
from preprocessBatch import preprocessing
from keras.models import load_model
import cv2
import numpy as np
import uuid  # Use uuid to generate unique file names

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    allowedChars = '234579ACFHKMNPQRTYZ'
    WIDTH = 140
    HEIGHT = 48

    # check if the post request has the file part
    if 'file' not in request.files:
        return jsonify(success=False, captcha="No image file part")

    file = request.files['file']

    # Create a unique filename using uuid
    filename = secure_filename(str(uuid.uuid4()))

    # Save file to temp path
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, filename)
    file.save(temp_path)

    img = Image.open(temp_path)
    img = img.resize((WIDTH, HEIGHT), Image.LANCZOS)
    # convert rgba to rgb
    img = img.convert('RGB')

    # Save processed image to temp path
    temp_path_processed = os.path.join(temp_dir, 'captcha.jpg')
    img.save(temp_path_processed, "JPEG")

    preprocessing(temp_path_processed, temp_path_processed)

    train_data = np.stack([np.array(cv2.imread(temp_path_processed))/255.0])
    model = load_model("thsrc_cnn_model.hdf5")
    prediction = model.predict(train_data)

    predict_captcha = ''
    for predict in prediction:
        value = np.argmax(predict[0])
        predict_captcha += allowedChars[value]

    # Remove temp files
    os.remove(temp_path)
    os.remove(temp_path_processed)

    return jsonify(success=True, captcha=predict_captcha)



start to process image from index: 0
completed


2024-02-12 10:50:09.793918: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-12 10:50:09.834515: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 10:50:09.834544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 10:50:09.837332: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-12 10:50:09.846623: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-12 10:50:09.847832: I tensorflow/core/platform/cpu_feature_guard.cc:1